This notebook will install numpy and matplotlib in your environment (the only 2 required packages).

The visualizer is in the last code block. An explanation is provided as well.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider

In [ ]:
t_max = 10
z_max = 10
samples = 1000
t = np.linspace(0, t_max, samples)
z = np.linspace(0, z_max, samples//2)
T, Z = np.meshgrid(t, z)

omega = 1
b = 2
phi_factor = 3
phi = np.pi/phi_factor
V = np.cos(omega*T - b*Z + phi)

In [ ]:
fig = plt.figure(figsize=(12,5))
ax_surf = fig.add_subplot(1, 2, 1, projection="3d")
ax_cross = fig.add_subplot(1, 2, 2)
ax_surf.set_xlabel("Time (s)")
ax_surf.set_ylabel("Distance (m)")
ax_surf.set_zlabel("Amplitude")
title = r"$V=\cos (\omega t - bz + \phi)$" + "\n" + fr"$\omega={omega}, b={b}, \phi=\pi/{phi_factor}$"
ax_surf.set_title(title)
ax_surf.view_init(elev=55, azim=-45)

ax_cross.set_xlabel("Distance (m)")
ax_cross.set_ylabel("Amplitude")
ax_cross.set_title("Snapshot in Time")

t_init = 0
z_init = phi/b
z_idx = np.argmin(np.abs(z-z_init))
ax_surf.plot_surface(T, Z, V, alpha=0.2)
plane_v = np.linspace(-1.5, 1.5, 2)
plane_z = np.linspace(0, z_max, 2)
PLANE_V, PLANE_Z = np.meshgrid(plane_v, plane_z)
PLANE_T = np.zeros_like(PLANE_V) + t[t_init]
surf_plane = [ax_surf.plot_surface(PLANE_T, PLANE_Z, PLANE_V, alpha=0.3, color="k")]
surf_cross, = ax_surf.plot(np.ones_like(z)*t[t_init], z, V[:, t_init], color="r")
surf_cross_space, = ax_surf.plot(t, np.ones_like(t)*z_init, V[z_idx, :], color="g")

cross_plot, = ax_cross.plot(z, V[:, t_init], color="r")

iso_line, = ax_surf.plot([0, t_max], [phi/b, phi/b + t_max*omega/b], [1, 1], color="k")
rise, = ax_surf.plot([2*np.pi/omega, 2*np.pi/omega], [phi/b, phi/b + 2*np.pi/b], [1, 1], color="k", linestyle="--", label="Spatial Period")
run, = ax_surf.plot([0, 2*np.pi/omega], [phi/b, phi/b], [1, 1], color="k", linestyle=":", label="Temporal Period")
dot, = ax_cross.plot(phi/b + t_init*omega/b, 1, "ko")

ax_surf.legend(loc="upper right")

axtime = fig.add_axes([0.25, 0, 0.65, 0.03])
time_slider = Slider(
    ax=axtime,
    label="Time (s)",
    valmin=0,
    valmax=t_max,
    valinit=t_init
)
def slider_update(val):
    time_idx = np.argmin(np.abs(t - val))
    z_val = phi/b + val*omega/b
    z_idx = np.argmin(np.abs(z - z_val))
    cross_plot.set_ydata(V[:, time_idx])
    dot.set_xdata([z_val])
    PLANE_T = np.zeros_like(PLANE_V) + t[time_idx]
    surf_plane[0].remove()
    surf_plane[0] = ax_surf.plot_surface(PLANE_T, PLANE_Z, PLANE_V, alpha=0.3, color="k")
    surf_cross.set_data_3d(np.ones_like(z)*t[time_idx], z, V[:, time_idx])
    surf_cross_space.set_data_3d(t, np.ones_like(t)*z[z_idx], V[z_idx, :])
    fig.canvas.draw_idle()

# register the update function with each slider
time_slider.on_changed(slider_update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.0, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset View', hovercolor='0.975')


def reset(event):
    ax_surf.view_init(elev=55, azim=-45)

button.on_clicked(reset)
plt.tight_layout()

# Explanation

The plot on the left shows the space-time surface of the wave with time on one axis and spatial distance on the other. The plot on the right shows the wave at a particular snapshot in time. This snapshot is equivalent to a cross-section of the space-time surface at a particular point in time (denoted by the plane in the space-time plot).

When the full space-time plot of the sinusoidal voltage wave is plotted in 3D, it's easy to track the velocity of the phase by looking at a particular peak as it evolves through time (the solid black line). This phase velocity, $u_p$, can simply be calculated by finding the slope (rise over run) which is defined by the spatial and temporal periods respectively. 

We know the spatial period, $\lambda$, is $\frac{2\pi}{\beta}$ while the temporal period, $T$, is $\frac{2\pi}{\omega}$. Dividing the two results in $u_p = \frac{\lambda}{T} = \frac{2\pi/\beta}{2\pi/\omega} = \frac{\omega}{\beta}$